In [1]:
import numpy as np
import os
import plyvel as ply
from PIL import Image, ImageOps
import StringIO
import gist
from sklearn.decomposition import PCA

def eigs_descend(X):
    eig_vals, eig_vecs = np.linalg.eig(X)
    # To get descending order, negate and sort in ascending order.
    sequence = np.argsort(-eig_vals)
    return eig_vecs[:, sequence]



def isoHash_lp(Lambda, iter, vector_size):
    a = np.trace(Lambda)/vector_size
    # Getting random orthogonal matrix U.
    R = np.random.random((vector_size, vector_size))
    U, _, _ = np.linalg.svd(R, compute_uv=1)
    Z = (U.dot(Lambda)).dot(U.T)

    for i in range(iter):
        # find T
        T = Z
        for j in range(vector_size):
            T[j, j] = a
        # find Z
        Q = eigs_descend(T)
        Z = (Q.dot(Lambda)).dot(Q.T)
    Q = Q.T
    return Q
def write_feature(db_path, converted_db_path,count, method='isohashlp',
    vector_size = 64, iter = 200):
    # A very primitive implementation. Need to refactor the code if
    # want to make it more versatile to a diversity of methods.
    if method.lower() == 'isohashlp':
        db = ply.DB(db_path)
        # I think we cannot modify in place. Have to create a new database.
        converted_db = ply.DB(converted_db_path, create_if_missing=True,
         write_buffer_size=268435456)
        # Assume we directly use the dimension 960 of gist. Can modify it later
        gist_dim = 960
        gists = np.zeros((gist_dim, count))
        i = 0
        with converted_db.write_batch() as converted_wb:
            for key, value in db:
                img = Image.frombytes('RGB', (224, 224), value)
                img_data = np.asarray(img)

                gistfeatures = gist.extract(img_data)
                # The 960-dimensional gists, serving as inputs for isoHash.
                # TODO: could have better implementation
                gists[:, i] = gistfeatures
                i += 1
                if i%100 == 0:
                    print i
            # zero center the data
            avg = np.sum(gists, axis=1)/count
            avg = avg.reshape((gist_dim, 1))
            gists = gists - avg
            print "before pca"

            # This PCA method is said to be inefficient. Can switch.
            pca = PCA(n_components = vector_size)
            gist_reduced = (pca.fit_transform(gists.T)).T
            print "after pca"
            # Each row is a Principal Component.
            # gist_reduced is W^T*X
            Lambda = np.dot(gist_reduced, gist_reduced.T)

            Q = isoHash_lp(Lambda, iter, vector_size)
            # TODO: Use sign function to get Y.
            Y = (Q.T).dot(gist_reduced)
            Y = (Y >= 0)*1
            
            i = 0
            for key, value in db:
               
                converted_key = Y[:, i].tobytes()
                converted_wb.put(converted_key, value)
                if i%100 == 0:
                    print i
                i = i + 1

In [3]:
def load_db(db_path, img_size=(224,224)):
    db = ply.DB(db_path, create_if_missing=True)
    imglist = []
    for key, value in db():
        imgitem = db.get(key)
        img = Image.frombytes('RGB',img_size,imgtest)
        imglist.append(img)

    return imglist

In [4]:
db_path = "/Users/rwa56/project/leveldb/"
converted_db_path="/Users/rwa56/project/isodb/"


In [ ]:
write_feature(db_path, converted_db_path,count=3000, method='isohashlp',
    vector_size = 64, iter = 200)

In [3]:
db = ply.DB(db_path, create_if_missing=True)

NameError: name 'ply' is not defined

In [2]:
imglist = []
for key, value in db:
        imgitem = db.get(key)
        img = Image.frombytes('RGB',(224,224),imgitem)
        imglist.append(img)


NameError: name 'db' is not defined

In [ ]:
gistfeatures = gist.extract(imglist[1])

In [ ]:
gist